# **Chapter 4: Separating Data from Instructions**

---
**Lesson:**

When using Llama2, you might not always want to write out full prompts from scratch. Instead, you can create prompt templates that you can later customize with specific data inputs before submitting to Llama2. This method is particularly useful when you want Llama2 to perform a consistent task, but with different data each time.

Creating prompt templates involves defining a fixed structure and identifying parts that can be replaced with variable user input. By doing this, you can quickly and efficiently generate customized prompts that Llama2 can execute.

In the following steps, we'll show you how to design a flexible prompt template and how to seamlessly integrate user-specific data into your prompts for Llama2.

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 512,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

In this first example (4.1), we're asking Llama2 to act as an animal noise generator. Notice that the full prompt submitted to Llama2 is just the prompt template substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the "{ANIMAL}" section.

> *Note: You don't have to call your substitution placeholder anything in particular in practice. For this example, definitely use {ANIMAL}, as that's how the exercise is formatted. But in general, just as easily, we could have called it "{CREATURE}" or "{A}" (but it's generally good to have your stand-ins be specific and relevant so that your prompt is easy to understand even without the substitution). Just make sure that whatever you name your substitution placeholder is what you use for the substitution formula."*

**Example 4.1 - Moo**

In [ ]:
from langchain import PromptTemplate
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""
    <s>[INST] 
    
    <<SYS>>
    <</SYS>>
    
    I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}.[/INST]"""
)

prompt = multi_var_prompt.format(ANIMAL="Cow")
    
response = invoke_model_and_get_response(prompt)
print(response)

**Example 4.2 - This could have been an email**

Why would we want to separate and substitute inputs like this? Well, prompt templates simplify repetitive tasks. Let's say you build a prompt structure that invites third party users to submit content to the prompt (in the previous example the animal whose sound they want to generate). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

We do this substitution here leverage an open source Python library called Langchain.  We use the {curly-brackets} formatting in our own code.

> *Note: Prompt templates can have as many variables as desired.*

When introducing substitution variables like this, it is very important to make sure Llama2 knows where variables start and end (vs. instructions or task descriptions). Let's look at an example where there is no separation between the instructions and the substitution variable.		

> *Note: We recommend that you use specifically XML tags as separators for Llama2, as opposed to some other separator, as Llama2's already trained to recognize XML tags in particular."*	

In [ ]:
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], 
    template="""
    <s>[INST] 
    
    <<SYS>>
    <</SYS>>
    
    Hey. Make this email more polite. <email> {EMAIL} </email>[/INST]"""
)

prompt = multi_var_prompt.format(EMAIL="Show up at 6am because I'm the CEO and I say so")
    
response = invoke_model_and_get_response(prompt)
print(response)

**Example 4.3 - The Farm**

Let's see another example of how XML tags can help us. 
We need to surround the user input sentences in XML tags. This shows Llama2 where the input data begins and ends despite the misleading hyphen before "Each is about an animal, like rabbits."

In [ ]:
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["SENTENCES"], 
    template="""
    <s>[INST] 
    
    <<SYS>>
    <</SYS>>
    
    I will give you a list of sentences. Each is about an animal, like rabbits. Tell me the second on the list. <sentences> {SENTENCES} </sentences>[/INST]"""
)

prompt = multi_var_prompt.format(SENTENCES="""
-I like how cows sound,
-This sentence may appear to be about dogs but it's actually about pigs,
-This sentence is about spiders""")
    
response = invoke_model_and_get_response(prompt)
print(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Excercise 4.1 - Haiku Topic**

Write a prompt in the highlighted template box that will take in a variable called "{TOPIC}" and output a haiku about the topic.

Remember to format everything properly, the way we covered in earlier lessons.

In [ ]:
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["TOPIC"], 
    template="""
    <s>[INST] 
    
    <<SYS>>
    <</SYS>>
    
    I will tell you a topic. Please write a haiku on that topic. <topic> {TOPIC} </topic>[/INST]"""
)

prompt = multi_var_prompt.format(TOPIC="Football")
    
response = invoke_model_and_get_response(prompt)
print(response)

**Excercise 4.2 - Dog Question With Typos**

Fix the prompt in the template below by adding XML tags, or making small edits so that Llama2 produces the right answer.

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Llama2 reacts to such mistakes.

In [ ]:
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["QUESTION"], 
    template="""
    <s>[INST] 
    
    <<SYS>>
    You are a helpful, respectful and honest assistant. 
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    <</SYS>>
    
    hia its me i have a q about dogs jkaerjv <question> {QUESTION} </question> jklmvca tx it help me muhch much atx fst fst answer short short tx [/INST]"""
)

prompt = multi_var_prompt.format(QUESTION="are cats brown?")
    
response = invoke_model_and_get_response(prompt)
print(response)

# Chapter 4 - END.